# swm-chatbot

Instalación dependencias

In [1]:
!pip install streamlit langchain langchain-google-genai langchain-community langchain-core langsmith langgraph


   ---------------------------------------- 0/5 [filetype]
  Attempting uninstall: google-auth
   ---------------------------------------- 0/5 [filetype]
    Found existing installation: google-auth 2.43.0
   ---------------------------------------- 0/5 [filetype]
    Uninstalling google-auth-2.43.0:
   ---------------------------------------- 0/5 [filetype]
      Successfully uninstalled google-auth-2.43.0
   ---------------------------------------- 0/5 [filetype]
   -------- ------------------------------- 1/5 [google-auth]
   -------- ------------------------------- 1/5 [google-auth]
   -------- ------------------------------- 1/5 [google-auth]
   -------- ------------------------------- 1/5 [google-auth]
   -------- ------------------------------- 1/5 [google-auth]
  Attempting uninstall: langchain-core
   -------- ------------------------------- 1/5 [google-auth]
    Found existing installation: langchain-core 1.2.0
   -------- ------------------------------- 1/5 [google-auth]
  

Crear una regla de firewall

In [2]:
%%bash
gcloud compute firewall-rules create allow-imc-streamlit \
    --direction=INGRESS \
    --priority=1000 \
    --network=default \
    --action=ALLOW \
    --rules=tcp:8502 \
    --source-ranges=0.0.0.0/0 2> /dev/null

gcloud compute firewall-rules list --filter name='allow-imc-streamlit' 2> /dev/null

NAME                 NETWORK  DIRECTION  PRIORITY  ALLOW     DENY  DISABLED
allow-imc-streamlit  default  INGRESS    1000      tcp:8502        False
NAME                 NETWORK  DIRECTION  PRIORITY  ALLOW     DENY  DISABLED
allow-imc-streamlit  default  INGRESS    1000      tcp:8502        False


Descargar archivos desde Google Drive

In [ ]:
%%bash
sudo pip install gdown --quiet

sudo gdown --fuzzy "https://drive.google.com/file/d/10Kx0tp-xBWYpfb1kB11bwvleQxGx_K-Y/view?usp=sharing" --output data/
sudo gdown --fuzzy "https://drive.google.com/file/d/1D_K45gpMkvAT5LMPZEc0xdp7M0UAObY4/view?usp=sharing" --output data/

sudo chown jupyter:jupyter -R data/

Activación de API de búsqueda

In [ ]:
%%bash
gcloud services enable apikeys.googleapis.com
gcloud services enable generativelanguage.googleapis.com
gcloud services list --enabled | grep -e 'apikeys' -e 'generative'

Variables del proyecto GCP

In [ ]:
import os

# --- Creación y Asignación de las variables
PROJECT_ID = !gcloud config get-value project 2> /dev/null
PROJECT_ID = PROJECT_ID[0].strip()

REGION = "us-central1"
ZONE = "us-central1-a"

INSTANCE_NAME = !gcloud compute instances list \
                   --filter='tags.items:("notebook-instance" "deeplearning-vm")' \
                   --format="value(name)"
INSTANCE_NAME = INSTANCE_NAME[0]

INSTANCE_NAME_IP = !gcloud compute instances describe $INSTANCE_NAME --zone=$ZONE --format='get(networkInterfaces[0].accessConfigs[0].natIP)'
INSTANCE_NAME_IP = INSTANCE_NAME_IP[0].strip()

MODEL_NAME = "gemini-2.5-flash-lite"

# --- Registro como variables de entorno
os.environ["PROJECT_ID"] = f"{PROJECT_ID}"
os.environ["REGION"] = REGION
os.environ["ZONE"] = ZONE
os.environ["INSTANCE_NAME"] = INSTANCE_NAME
os.environ["INSTANCE_NAME_IP"] = INSTANCE_NAME_IP
os.environ["MODEL_NAME"] = f"{MODEL_NAME}"

# --- Visualización de las variables
print(f"PROJECT_ID: {PROJECT_ID}")
print(f"REGION: {REGION}")
print(f"ZONE: {ZONE}")
print(f"INSTANCE_NAME: {INSTANCE_NAME}")
print(f"INSTANCE_NAME_IP: {INSTANCE_NAME_IP}")
print(f"MODEL_NAME: {MODEL_NAME}")

Creación de la aplicación

In [3]:
%%writefile swm-chat.py
import streamlit as st
import os
import warnings

from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage

from langchain_community.document_loaders import PyPDFLoader
# ----------------------------------
#  Configuración
# ----------------------------------

# --- Configuración de la página
st.set_page_config(page_title="Chatbot con LangChain", page_icon="⚙️", layout="centered")

# --- Configuración del nivel de logs
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# --- Lectura de la API Key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    st.error("❌ Error: Falta la API Key")
    st.stop()

# --- Configuración de variables de entorno
PROJECT_ID = os.environ.get("PROJECT_ID")
MODEL_NAME = os.environ.get("MODEL_NAME")

#foundation_models = ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro"]
#model_name = st.selectbox("Models:", foundation_models)
#if "current_model" not in st.session_state:
#    st.session_state.current_model = model_name

contexto = """
Eres un asistente virtual que ayuda a nadadores a obtener información sobre el mundo de la natación,
como por ejemplo extraer datos de las normativas de las competiciones, records y marcas personales de otros nadadores o
noticias generales relacionadas. Es importante solo responder a consultas relacionadas con la natación.
"""

#hacer una para minimas de España (leer pdf) y otra para records (leer página/API)
#para leer pdfs igual interesa meterlo en drive
#se puede probar la API de youtube para buscar cosas de fastskins

"""
@tool
def buscar_minimas(query: str) -> str:
    archivos_pdf = [f for f in os.listdir('data') if f.endswith('.pdf')]

    if archivos_pdf:
        path = os.path.join('data', archivos_pdf[0])
        loader = PyPDFLoader(path)

        # Esto carga el PDF y lo divide por páginas automáticamente
        documentos = loader.load()

        print(f"Se cargaron {len(documentos)} páginas.")
        print("Contenido de la página 1:")
        print(documentos[0].page_content)
    else:
        print("No se encontraron PDFs en la carpeta data/") """

@tool
def nivel_natacion_libres(metros: int, minutos: int) -> str:
    if metros == 50:
            if minutos > 1:
                return "nivel municipal"
            elif minutos < 1:
                return "nivel nacional"
            else:
                return "nivel regional"
    else:
        return "muchos metros"


tools = [nivel_natacion_libres]

# ----------------------------------
#  Configuración del Agente
# ----------------------------------
@st.cache_resource
def iniciar_agente():
    try:
        llm = ChatGoogleGenerativeAI(
            model=MODEL_NAME,
            google_api_key=GOOGLE_API_KEY,
            temperature=0,
            # -- Conversión del System Prompt en algo que Gemini respeta
            convert_system_message_to_human=True
        )
        return create_react_agent(llm, tools=tools)
    except Exception as e:
        estado = f"🔴 Error: {str(e)}"
        model = None

estado = f"🟢 Conectado a {PROJECT_ID} ({MODEL_NAME}) - LangChain (Chain of Thought)"
agent = iniciar_agente()

# ----------------------------------
# Diseño de la interfaz de usuario
# ----------------------------------
st.title("🎓 Chat con LangChain-Agent")
st.caption(estado)

if "messages" not in st.session_state:
    st.session_state.messages = []

# Renderizar historial previo
for msg in st.session_state.messages:
    if isinstance(msg, (HumanMessage, AIMessage)):
        role = "user" if isinstance(msg, HumanMessage) else "assistant"
        with st.chat_message(role):
            st.markdown(msg.content)

# -- Input de Usuario
if prompt := st.chat_input("Ej: Habitantes de Pelayos"):

    # -- Escritura del mensaje de usuario
    st.session_state.messages.append(HumanMessage(content=prompt))
    with st.chat_message("user"):
        st.markdown(prompt)

    # -- Lógica del Agente (Asistente)
    with st.chat_message("assistant"):

        # -- Variables para guardar el resultado y usarlo fuera
        response = None
        texto_final = ""

        # ---------------------------------------------------------
        # Visualización (Bloque de Pensamiento
        # ---------------------------------------------------------
        with st.status("🧠 Analizando solicitud...", expanded=True) as status:
            try:
                # -- Preparación del input (prompt de entrada)
                sys_msg = SystemMessage(content=prompt_template)
                mensajes_input = [sys_msg] + st.session_state.messages

                # -- Ejecución del Agente
                response = agent.invoke({"messages": mensajes_input})

                # -- Visualización de pasos intermedios
                mensajes_totales = response["messages"]
                nuevos_mensajes = mensajes_totales[len(mensajes_input):]
                herramienta_usada = False

                for msg in nuevos_mensajes:
                    if isinstance(msg, AIMessage) and msg.tool_calls:
                        for call in msg.tool_calls:
                            st.write(f"🛠️ **Decisión:** Ejecutar `{call['name']}`")
                            herramienta_usada = True
                    elif isinstance(msg, ToolMessage):
                        st.write("💾 **Datos obtenidos:**")
                        st.code(msg.content, language="json")

                # Cierre de la caja de Análisis (Pensamiento)
                if herramienta_usada:
                    status.update(label="✅ Datos obtenidos correctamente", state="complete", expanded=False)
                else:
                    status.update(label="ℹ️ Respuesta sin uso de herramientas", state="complete", expanded=False)

            except Exception as e:
                status.update(label="❌ Error", state="error")
                st.error(f"Error: {e}")
                st.stop()

        # ---------------------------------------------------------
        #  Respuesta Final
        # ---------------------------------------------------------
        if response:
            ultimo_mensaje = response["messages"][-1]
            raw_content = ultimo_mensaje.content

            # --- Limpieza de Gemini (para devolución de listas)
            if isinstance(raw_content, list):
                 texto_final = "".join([x.get("text", "") for x in raw_content if isinstance(x, dict)])
            else:
                 texto_final = str(raw_content)

            # -- Impresión en el Chat Principal
            st.markdown(texto_final)

            # -- Almacenamiento den el historial
            st.session_state.messages.append(AIMessage(content=texto_final))

Writing swm-chat.py


In [ ]:
!streamlit run swm-chat.py --server.port 8502




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://10.150.0.3:8502
  External URL: http://34.48.165.177:8502

/opt/conda/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
